# TFY4345 - Classical Mechanics - Numerical Exercise


In [ ]:
'''Imports packages, functions and constants'''
# Simple pendulum with Eulers method:
import math
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.animation import FuncAnimation
from scipy.constants import g
from scipy.integrate import solve_ivp
from IPython.display import HTML

In [ ]:
'''Total energy of a pendulum'''
def energies(theta, omg):
    T = (1/2)*m*(l**2)*omg**2
    V = m*g*l*(1 - np.cos(theta))
    E_tot = T + V
    return T, V, E_tot

## Simple Pendulum

### Analytical solution with smal angle aproximation (harmonic oscilator)


For comparison's sake, n stuff, ellerno

Plot some sin stuff

### Eulers Method

In [ ]:
'''Function for Eulers method'''
def euler_step(theta_vec, omg_vec, i, dt):
    omg_vec[i+1] = omg_vec[i] - (g/l)*np.sin(theta_vec[i])*dt
    theta_vec[i+1] = theta_vec[i] + omg_vec[i]*dt

def euler_solve(theta_vec, omg_vec, dt):
    for i in range(timesteps-1):
        euler_step(theta_vec, omg_vec, i, dt)

In [ ]:
'''Parameters and initial conditiions'''
theta_0 = 0.2 #rad
omg_0 = 0 #rad/s
l = 1.0 #m (meter)
m = 1.0 #kg
sim_time = 10 #s, simulation time
dt = 0.01 # s, timestep
timesteps = int(sim_time/dt)

In [ ]:
'''arrays with vals for plotting'''
omg_vec = np.zeros(timesteps)
theta_vec = np.zeros(timesteps)
#t_vec = np.linspace(0, timesteps, theta_vec.size)
t_vec = np.linspace(0, sim_time, timesteps)

# set initial values
omg_vec[0] = omg_0
theta_vec[0] = theta_0

In [ ]:
euler_solve(theta_vec, omg_vec, dt)
kin_energy, pot_energy, total_energy = energies(theta_vec, omg_vec)

In [ ]:
# plot results
plt.plot(t_vec, theta_vec)
plt.show()
plt.plot(t_vec, omg_vec)
plt.show()

plt.plot(t_vec, kin_energy)
plt.plot(t_vec, pot_energy)
plt.plot(t_vec, total_energy)
plt.show()

### Euler-Cromer method

In [ ]:
def euler_cromer_step(theta_vec, omg_vec, i, dt):
    omg_vec[i+1] = omg_vec[i] - (g/l)*np.sin(theta_vec[i])*dt
    theta_vec[i+1] = theta_vec[i] + omg_vec[i+1]*dt

def euler_cromer_solve(theta_vec, omg_vec, dt):
    for i in range(timesteps-1):
        euler_cromer_step(theta_vec, omg_vec, i, dt)

In [ ]:
'''New arrays for Euler-Cromer'''
omg_vec_cromer = np.zeros(timesteps)
theta_vec_cromer = np.zeros(timesteps)

# set initial values
omg_vec_cromer[0] = omg_0
theta_vec_cromer[0] = theta_0

In [ ]:
euler_cromer_solve(theta_vec_cromer, omg_vec_cromer, dt)
kin_energy_cromer, pot_energy_cromer, total_energy_cromer = energies(theta_vec_cromer, omg_vec_cromer)

In [ ]:
# plot results
plt.plot(t_vec, theta_vec_cromer)
plt.show()

plt.plot(t_vec, omg_vec_cromer)
plt.show()

plt.plot(t_vec, kin_energy_cromer)
plt.plot(t_vec, pot_energy_cromer)
plt.plot(t_vec, total_energy_cromer)
plt.show()

In [ ]:
'''Animation, currently at arbitrary speed/time :p'''

# cartesian coordinates of the pendulum tip
x = np.sin(theta_vec_cromer)
y = -np.cos(theta_vec_cromer)

# arbitrarily chosen FPS
FPS = 30

# Set up fig and stuff for the animated plot
fig = plt.figure()
ax = plt.axes(xlim=(-2, 2), ylim=(-2, 2))
string, = ax.plot([], [], lw=2)
mass = plt.Circle((x[0], y[0]), 0.05, color='blue')
framecount = int(FPS*sim_time)

def init():
    string.set_data([],[])
    ax.add_artist(mass)
    return string,


def animate(i):
    mass.center = (x[i], y[i])
    string.set_data([0, x[i]], [0, y[i]])
    return string, mass


anim = FuncAnimation(fig, animate, init_func=init, frames=framecount, interval=1000*dt, blit=True)
HTML(anim.to_html5_video())


### Runge-Kutta 4th order

In [ ]:
def ddt_omg_theta(t, y):
    return [y[1], -(g/l)*np.sin(y[0])]

In [ ]:
sol = solve_ivp(ddt_omg_theta, [0, 10], [0.2, 0])
kin_energy_RK45, pot_energy_RK45, total_energy_RK45 = energies(sol.y[0], sol.y[1])

In [ ]:
plt.plot(sol.t, sol.y[0])
plt.show()
plt.plot(sol.t, sol.y[1])
plt.show()

plt.plot(sol.t, kin_energy_RK45)
plt.plot(sol.t, pot_energy_RK45)
plt.plot(sol.t, total_energy_RK45)
plt.show()